Este é meu primeiro projeto de Data Science com web crawling. Quero prever o preço de carros usando dados do site webmotors.
Vou usar vários modelos e comparar os resultados deles .

O bloco a baixo faz o web scrapping do site WebMotors. Porém o site é dinâmico o que traz dificuldades. Usei o selenium para abrir o site e rolar até alcançar o final da página. Quando ele chega no final da página o beautifulsoup abre o código fonte, extraí os links e salva os links únicos (somente de carros) em um arquivo csv.
Não recomendo rodar esse bloco, é bem demorado. No repositório já tem o arquivo csv com todos os links pegos.

In [2]:
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
import csv

driver = webdriver.Chrome()
driver.get('https://www.webmotors.com.br/carros-usados/estoque')
time.sleep(0.1)
element = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/button')
element.click()
urls = open('urls2.csv', mode='w')
url_writer = csv.writer(urls, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
heights = []
links=[]
counter = 0
for i in range(1,10000):
    bg = driver.find_element_by_css_selector('body')
    time.sleep(0.1)
    bg.send_keys(Keys.END)
    heights.append(driver.execute_script("return document.body.scrollHeight"))
    try :
        bottom = heights[i-16]
    except:
        pass
    if i%16 ==0:
        new_bottom = heights[i-1]
        if bottom == new_bottom:
            try:
                element = driver.find_element_by_xpath('/html/body/div/main/div[1]/div[3]/div[3]/div/div/button')
                element.click()
                soup = BeautifulSoup(driver.page_source, 'html')
                for link in soup.findAll('a'):
                    check_str = 'https://www.webmotors.com.br/comprar/'
                    link_string = str(link.get('href'))
                    res = check_str in link_string
                    if link_string not in links and res == True:
                        links.append(link_string)
                        url_writer.writerow([link_string])

            except:
                break

urls.close()

Nessa próxima parte vamos tirar as caracteristícas de cada carro e organiza-las nas colunas 'modelo', 'combustível', 'ano','quilometragem', 'final de placa', 'carroceria' e 'cor'.

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
import time
from lxml import etree

urls = open('urls.csv', mode='r')
carros = open('carros.csv', mode='w')
carros_writer = csv.writer(carros, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
carros_writer.writerow(['modelo' , 'combustivel' , 'ano' , 'quilometragem' , 'final de placa' , 'carroceria' , 'cor'])
for row in urls:
    driver = webdriver.Chrome()
    driver.get(row,)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html')
    dom = etree.HTML(str(soup))

    modelo = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[1]/h1/strong')[0].text
    combs = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[5]/strong')[0].text
    ano = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[1]/strong')[0].text
    km = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[2]/strong')[0].text
    placa = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[6]/strong')[0].text
    carro = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[4]/strong')[0].text
    cor = dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li[7]/strong')[0].text
    print(dom.xpath('/html/body/div/main/div[2]/div[1]/div/div[2]/div/ul/li[3]/h3')[0].text)

    for i in range(1,100):
        try: 
            feature = dom.xpath(('/html/body/div/main/div[2]/div[1]/div/div[2]/div/ul/li[{0}]/h3').format(i))[0].text
            if feature == 'Ar condicionado':
                arcondicionado = 'sim'
            if feature == 'Direção hidráulica' or feature == 'Direção elétrica' or feature == 'direção mecânica':
                direcao = feature
        except:
            break




   
    for item in soup.findAll('h3'):
        print(str(item.get('class')))
        if str(item.get('href')) == 'Ar condicionado':
            arcondicionado = 'sim'
        else:
            arcondicionado = 'nao'

    carros_writer.writerow( [ [modelo] , [combs] , [ano] , [km] , [placa] , [carro] , [cor], [arcondicionado] ] )
    print([modelo] , [combs] , [ano] , [km] , [placa] , [carro] , [cor], [arcondicionado])
    driver.quit()



WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


Neste bloco vou analisar os dados adquiridos.